In [1]:
import pandas as pd
import Clean
import numpy as np

In [2]:
offers = pd.read_csv("offer_acceptance_offers.csv",low_memory=False)
orders = pd.read_csv('offer_acceptance_orders.csv',low_memory=False)
orders = Clean.clean_orders(orders)
offers = Clean.clean_offers(offers)

In [3]:
# create FTL Orders and Joined DataFrame with FTL Order to Offers
ftl = orders[orders['TRANSPORT_MODE'] == 'FTL']#.head(10000)
join = ftl.set_index('REFERENCE_NUMBER').join(offers.set_index('REFERENCE_NUMBER'),how = 'inner')

In [4]:
# making sure all offers are quote
print(join[join['OFFER_TYPE'] == 'quote'].shape)
print(join.shape)

(269354, 30)
(269354, 30)


In [5]:
# create data set with only FTL offers and get a list of unique FTL Offers
ftlOffers = join[['CARRIER_ID','CREATED_ON_HQ','RATE_USD']].sort_values('CREATED_ON_HQ')
offersRefNumList = np.unique(ftlOffers.index)
len(offersRefNumList)

68817

In [6]:
ftl.set_index('REFERENCE_NUMBER',inplace = True)

In [7]:
ftl['NumberOffers'] = ftlOffers.groupby(level=0).count()['CARRIER_ID']//np.e
ftl.dropna(inplace = True)

/tmp/ipykernel_407/1483909289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ftl['NumberOffers'] = ftlOffers.groupby(level=0).count()['CARRIER_ID']//np.e
/tmp/ipykernel_407/1483909289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ftl.dropna(inplace = True)


In [8]:
ftl.reset_index(inplace = True)

In [85]:
refNums = []
carrierID = []
rate = []
# loop through unique ref numbers
for offerRef in offersRefNumList:
    # get the order that goes to a refrence number
    order = ftl.loc[ftl['REFERENCE_NUMBER'] == offerRef]
    #offer
    arr = ftlOffers.loc[offerRef]
    rates = np.array(arr['RATE_USD'])
    carrier = np.array(arr['CARRIER_ID'])
    secValue = order['NumberOffers'].reset_index()['NumberOffers'][0]
    print(secValue)
    value2beat = np.min(rates[:int(secValue)])
    for i in range(len(rates[int(secValue):])):
        if rates[i] > value2beat:
            rate.append(rates[i])
            carrierID.append(carrier[i])
            refNums.append(offerRef)
            break           

0.0


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [72]:
len(refNums)

0

In [35]:
refNums = []
carrierIDs = []
rates = []
# loop through unique ref numbers
for offerRef in offersRefNumList:
    # get the order that goes to a refrence number
    order = ftl.loc[ftl['REFERENCE_NUMBER'] == offerRef]
    # get the offer as an array
    offer = ftlOffers.loc[offerRef].values.tolist()
    cost = order['ESTIMATED_COST_AT_ORDER'].reset_index()['ESTIMATED_COST_AT_ORDER'][0] * 1.5
    if isinstance(offer[0],str):
        if cost > offer[2]:
            refNums.append(offerRef)
            carrierIDs.append(offer[0])
            rates.append(offer[2])
    else:
        for ind in offer:
            if cost > ind[2]:
                refNums.append(offerRef)
                carrierIDs.append(ind[0])
                rates.append(ind[2])
                break
            

In [36]:
len(refNums)

67954

In [42]:
rates

[3483.0,
 5676.0,
 3096.0,
 9288.0,
 774.0,
 5095.5,
 1677.0,
 6579.0,
 6450.0,
 1548.0,
 1548.0,
 3870.0,
 2709.0,
 645.0,
 1419.0,
 2709.0,
 5160.0,
 3870.0,
 851.4,
 1096.5,
 9030.0,
 1096.5,
 903.0,
 5547.0,
 3870.0,
 3612.0,
 3225.0,
 8772.0,
 2580.0,
 5547.0,
 1677.0,
 9675.0,
 1677.0,
 645.0,
 2902.5,
 202.53,
 451.5,
 2451.0,
 967.5,
 1096.5,
 5418.0,
 3225.0,
 903.0,
 1108.11,
 645.0,
 6966.0,
 4128.0,
 6579.0,
 967.5,
 161.25,
 10320.0,
 1161.0,
 5805.0,
 757.0881,
 1032.0,
 1290.0,
 3870.0,
 2580.0,
 2322.0,
 3354.0,
 2902.5,
 2386.5,
 2451.0,
 2193.0,
 9288.0,
 9675.0,
 903.0,
 4773.0,
 6192.0,
 3096.0,
 5805.0,
 2064.0,
 4902.0,
 774.0,
 6708.0,
 1806.0,
 12688.1175,
 4386.0,
 2193.0,
 4515.0,
 7095.0,
 3225.0,
 1251.3,
 1935.0,
 5031.0,
 1419.0,
 6450.0,
 2322.0,
 4773.0,
 709.5,
 1161.0,
 7224.0,
 4773.0,
 1741.5,
 2322.0,
 1677.0,
 5160.0,
 4515.0,
 2386.5,
 3612.0,
 2322.0,
 1548.0,
 10578.0,
 1161.0,
 1419.0,
 4128.0,
 2580.0,
 1806.0,
 2354.25,
 8385.0,
 6450.0,
 341

In [44]:
results = pd.DataFrame({'refNums':refNums,'Carriers':carrierIDs,'Rate':rates})
results

,refNums,Carriers,Rate
0,"[\n ""000020adb97f32cf89e5f094b35607499f7ff872...",6326cfe26285d0f687c2cc5385aad8a0f7b2664a30c84a...,3483.0
1,"[\n ""0002e80902489c5a5eed029971f32e2ade351387...",f9b016cc00657ed194e66a0988a6ace5188a4ed5adc976...,5676.0
2,"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176...",ccf51d90eb2019002bd4b6f104a5c4dec9aa4fc2fb3672...,3096.0
3,"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa22...",c25ed90a126cbdabb004b777139973b0652f84051907c8...,9288.0
4,"[\n ""0004ef769b6e6e2a753c030ba14bfb33c88e392c...",277cdf8be8cde82af8e7b5a885d874432c7703a0bb062a...,774.0
...,...,...,...
67949,"[\n ""fffb240d62d732bd179440f15272283f408368ed...",fdbcf987fae167738b6002cf6ffe0581b553cc430e284c...,7740.0
67950,"[\n ""fffbec3e1ffcf0c505877a2b68e8fc773a413f9d...",bc8db39f614342b78a67494dbece216d3726f6924b7356...,9417.0
67951,"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97...",b910260d6ccd1fedb9919e9974c4b1eacf7ff1e3a4643c...,6708.0
67952,"[\n ""fffdb6cd77e2d5e8bfa5a355cf80a2c90a1a800f...",50239c17d7eb7a2a950453da494081501410231b48d7f4...,3225.0
